In [31]:
import pandas as pd

In [32]:
allergies = pd.read_csv('allergies.csv')
careplans = pd.read_csv('careplans.csv')
claims = pd.read_csv('claims.csv')
conditions = pd.read_csv('conditions.csv')
encounters = pd.read_csv('encounters.csv')
immunizations = pd.read_csv('immunizations.csv')
medications = pd.read_csv('medications.csv')
observations = pd.read_csv('observations.csv')
patients = pd.read_csv('patients.csv')

Size of indiv. dataframes
- delete columns that we are not going to use before we merge

- read in datasets a chunk at a time
    - operate on sequential parts of the data

- if its still an issue, could get set up on cluster to compute there as well

In [34]:
column_names = ['START','STOP']
allergies_clean = allergies.drop(column_names, axis=1)
careplans_clean = careplans.drop(column_names, axis=1)
conditions_clean = conditions.drop(column_names, axis=1)
medications_clean = medications.drop(column_names, axis=1)

# Drop unnecessary patient information
patients = patients.drop('ssn', axis=1)
patients = patients.drop('passport', axis=1)
patients = patients.drop('first', axis=1)
patients = patients.drop('last', axis=1)
patients = patients.drop('maiden', axis=1)
patients = patients.drop('suffix', axis=1)
patients = patients.drop('prefix', axis=1)
## Do we need address/birthplace? could be geographically interesting ig?
patients = patients.drop('drivers', axis=1) ## WHAT IS DRIVERS? don't think we need but do more research to make sure

# drop diagnosis from claims as all entries are empty
claims = claims.drop('DIAGNOSIS', axis=1)
claims = claims.drop('ORGANIZATION', axis=1) # deleted this as all organizations are temp organizations

column_names = ['DATE']
observations_clean = observations.drop(column_names, axis=1)
immunizations_clean = immunizations.drop(column_names, axis=1)
encounters_clean = encounters.drop(column_names, axis=1)

## should we remove descriptions as we will prob use codes to train model and descriptions only as a way to understand the model's output?

In [35]:
patients = patients.rename(columns={'patient': 'PATIENT'})

def add_suffix(df, suffix):
    renamed_columns = {}
    for col_name in df.columns:
        if col_name != 'PATIENT':
            renamed_columns[col_name] = col_name + '_' + suffix
        else:
            renamed_columns[col_name] = col_name
    return df.rename(columns=renamed_columns)


allergies_clean = add_suffix(allergies_clean, 'ALLERGIES')
careplans_clean = add_suffix(careplans_clean, 'CAREPLANS')
claims_clean = add_suffix(claims, 'CLAIMS')
conditions_clean = add_suffix(conditions_clean, 'CONDITIONS')
encounters_clean = add_suffix(encounters_clean, 'ENCOUNTERS')
immunizations_clean = add_suffix(immunizations_clean, 'IMMUNIZATIONS')
medications_clean = add_suffix(medications_clean, 'MEDICATIONS')
observations_clean = add_suffix(observations_clean, 'OBSERVATIONS')


In [36]:
merged_df = pd.merge(allergies_clean, patients, on='PATIENT', how='outer')
merged_df = pd.merge(merged_df, careplans_clean, on='PATIENT', how='outer')

# merged_df

In [37]:
merged_df = pd.merge(merged_df, claims_clean, on='PATIENT', how='outer')
merged_df = pd.merge(merged_df, conditions_clean, on='PATIENT', how='outer')

In [8]:
#merged_df = pd.merge(merged_df, encounters_clean, on='PATIENT', how='inner')

In [9]:
# merged_df

In [38]:
total_rows = len(encounters_clean)
rows_per_section = total_rows // 8

section1 = encounters_clean.iloc[:rows_per_section]
section2 = encounters_clean.iloc[rows_per_section:2*rows_per_section]
section3 = encounters_clean.iloc[2*rows_per_section:3*rows_per_section]
section4 = encounters_clean.iloc[3*rows_per_section:4*rows_per_section]
section5 = encounters_clean.iloc[4*rows_per_section:5*rows_per_section]
section6 = encounters_clean.iloc[5*rows_per_section:6*rows_per_section]
section7 = encounters_clean.iloc[6*rows_per_section:7*rows_per_section]
section8 = encounters_clean.iloc[7*rows_per_section:]

# print(section1.size)
# print(section2.size)
# print(section3.size)
# print(section4.size)
# print(section5.size)
# print(section6.size)
# print(section7.size)
# print(section8.size)
# print(total_rows)

section1 = add_suffix(section1, '1')
section2 = add_suffix(section2, '2')
section3 = add_suffix(section3, '3')
section4 = add_suffix(section4, '4')
section5 = add_suffix(section5, '5')
section6 = add_suffix(section6, '6')
section7 = add_suffix(section7, '7')
section8 = add_suffix(section8, '8')

In [39]:
merged_df = pd.merge(merged_df, section1, on='PATIENT', how='outer')

In [12]:
# merged_df

In [13]:
# section2

In [40]:
merged_df = pd.merge(merged_df, section2, on='PATIENT', how='outer')

In [15]:
# section3["PATIENT"]

In [41]:
merged_df = pd.merge(merged_df, section3, on='PATIENT', how='outer')

In [42]:
merged_df = pd.merge(merged_df, section4, on='PATIENT', how='outer')

: 

In [ ]:
merged_df = pd.merge(merged_df, section5, on='PATIENT', how='outer')

In [ ]:
merged_df = pd.merge(merged_df, section6, on='PATIENT', how='outer')

In [ ]:
merged_df = pd.merge(merged_df, section7, on='PATIENT', how='outer')

In [ ]:
merged_df = pd.merge(merged_df, section8, on='PATIENT', how='outer')

In [9]:
merged_df = pd.merge(final_merged_df, immunizations_clean, on='PATIENT', how='outer')

In [10]:
merged_df = pd.merge(merged_df, medications_clean, on='PATIENT', how='outer')
merged_df = pd.merge(merged_df, observations_clean, on='PATIENT', how='outer')

: 